In [ ]:
!pip install langchain langchain-community langchain-core langchain-groq langchain-huggingface pypdf faiss-cpu sentence-transformers requests

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-groq to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-huggingface to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installat

In [ ]:
import os
os.environ["GROQ_API_KEY"] = "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"

In [ ]:
import os
from dotenv import load_dotenv
import requests
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.agents import Tool, initialize_agent
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import MessagesPlaceholder



In [ ]:
pdf_url="https://arxiv.org/pdf/2207.13050.pdf"
pdf_file = "my.pdf"
if not os.path.exists(pdf_file):
    pdf = requests.get(pdf_url)
    with open(pdf_file, "wb") as f:
        f.write(pdf.content)


In [ ]:
loader = PyPDFLoader(pdf_file)
documents = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size = 3000, chunk_overlap = 450)
chunks = splitter.split_documents(documents)

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(chunks, embeddings)
retriever = vector_store.as_retriever()


In [ ]:
llm=ChatGroq(model="openai/gpt-oss-20b", temperature=0)
qa_chain = ConversationalRetrievalChain.from_llm(llm, retriever)
tools = [
    Tool(
        name="PDF_QA",
        func=lambda q: qa_chain.invoke({"question": q, "chat_history": []})["answer"],
        description="Use this to answer questions based on the loaded PDF."
    )
]


In [ ]:

chat_history = ChatMessageHistory()

memory = ConversationBufferMemory(
    memory_key="chat_history",
    chat_memory=chat_history,
    return_messages=True
)

In [ ]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="chat-conversational-react-description",
    memory=memory,
    verbose=False,
    system_message=(
        "You are an AI assistant that MUST answer questions using the tool 'PDF_QA' "
        "which retrieves information from the research paper. "
        "You are NOT allowed to answer from your general knowledge. "
        "Every answer must clearly say: 'Tool used: PDF_QA'. "
        "If the tool does not provide an answer, say 'The PDF does not contain this information'."
    )
)

In [ ]:
from IPython.display import display, Markdown, clear_output

chat_history = []


In [ ]:
while True:
    question = input("You: ").strip()
    if question.lower() == "exit":
        print("Goodbye ")
        break
    if not question:
        continue

    # Add user message
    chat_history.append(("User", question))

    # Get model response
    response = agent.invoke({"input": question})
    answer = response["output"]

    # Add bot message
    chat_history.append(("Bot", answer))

    # Clear previous output to simulate chat flow
    clear_output(wait=True)

    # Display the full conversation cleanly
    for role, msg in chat_history:
        if role == "User":
            display(Markdown(f"** You:** {msg}"))
        else:
            display(Markdown(f"** Bot:** {msg}"))

    print("\n(Type 'exit' to stop)")








You: exit
Goodbye 
